In [1]:
pip install langchain_openai langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing insta

In [2]:
import requests
import numpy as np
import json
from scipy.interpolate import interp1d
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def fetch_windborne_data(hours=24):
    all_data = {}
    for h in range(hours):
        url = f"https://a.windbornesystems.com/treasure/{h:02d}.json"
        response = requests.get(url)
        if response.status_code == 200:
            try:
                data = response.json()
                if isinstance(data, list) and all(isinstance(item, list) and len(item) == 3 for item in data):
                    all_data[h] = np.array(data)
                else:
                    print(f"Warning: Skipping malformed data from {url}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {url}: {e}")
        else:
            print(f"Failed to fetch {url}, status code: {response.status_code}")

    if not all_data:
        print("No valid data collected.")
        return None
    
    # Ensure data is structured as (hours, balloons, coordinates) only for existing hours
    num_balloons = max(arr.shape[0] for arr in all_data.values())
    structured_data = np.full((len(all_data), num_balloons, 3), np.nan)
    
    for i, (h, data) in enumerate(all_data.items()):
        structured_data[i, :data.shape[0], :] = data
    
    # Fill missing values with column mean
    for d in range(3):  # Iterate over x, y, z coordinates
        valid_mask = ~np.isnan(structured_data[:, :, d])
        if valid_mask.any():  # If at least one valid value exists
            mean_value = np.nanmean(structured_data[:, :, d])
            structured_data[:, :, d] = np.where(np.isnan(structured_data[:, :, d]), mean_value, structured_data[:, :, d])
    
    return structured_data

all_positions = fetch_windborne_data()

if all_positions is not None:
    print("Data Shape (after full interpolation):", all_positions.shape)
    print("Any NaN remaining?", np.isnan(all_positions).any())
else:
    print("No valid data available.")

Failed to fetch https://a.windbornesystems.com/treasure/05.json, status code: 404
Error decoding JSON from https://a.windbornesystems.com/treasure/06.json: Extra data: line 6 column 6 (char 93)
Error decoding JSON from https://a.windbornesystems.com/treasure/07.json: Extra data: line 6 column 6 (char 94)
Failed to fetch https://a.windbornesystems.com/treasure/08.json, status code: 404
Failed to fetch https://a.windbornesystems.com/treasure/09.json, status code: 404
Failed to fetch https://a.windbornesystems.com/treasure/10.json, status code: 404
Failed to fetch https://a.windbornesystems.com/treasure/11.json, status code: 404
Failed to fetch https://a.windbornesystems.com/treasure/14.json, status code: 404
Error decoding JSON from https://a.windbornesystems.com/treasure/15.json: Extra data: line 6 column 6 (char 92)
Failed to fetch https://a.windbornesystems.com/treasure/16.json, status code: 404
Failed to fetch https://a.windbornesystems.com/treasure/17.json, status code: 404
Failed t

In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score

def process_sequences(data, time_steps=5):
    """Prepare LSTM sequences for training, ensuring correct order."""
    num_balloons = data.shape[1]
    X, y = [], []
    scalers = [MinMaxScaler() for _ in range(num_balloons)]

    for i in range(num_balloons):
        balloon_data = data[:, i, :][::-1]  # Reverse order: [08, 07, ..., 00]
        balloon_data = scalers[i].fit_transform(balloon_data)

        # Generate time-step sequences
        for j in range(len(balloon_data) - time_steps):
            X.append(balloon_data[j:j+time_steps])
            y.append(balloon_data[j+time_steps])

    return np.array(X), np.array(y), scalers

# Prepare LSTM sequences
time_steps = 5
X, y, scalers = process_sequences(all_positions, time_steps)
print("LSTM Input Shape:", X.shape)  # Expected: (num_samples, time_steps, 3)

# Define Early Stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Define LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, activation="relu", input_shape=(time_steps, 3)),
    LSTM(32, return_sequences=False, activation="relu"),
    Dense(16, activation="relu"),
    Dense(3)  # Predicts (lat, lon, alt)
])

model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss="mse")

# Train Model
history = model.fit(X, y, epochs=30, batch_size=16, validation_split=0.1, callbacks=[early_stop])

y_pred = model.predict(X)
mse = mean_squared_error(y.reshape(-1, 3), y_pred.reshape(-1, 3))
r2 = r2_score(y.reshape(-1, 3), y_pred.reshape(-1, 3))

print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"R² Score: {r2:.6f}")

def predict_next_position(model, all_positions, scalers, time_steps=5):
    """Predict the next step using the latest available data from each balloon."""
    num_balloons = len(scalers)
    predicted_next_real = []

    for i in range(num_balloons):
        balloon_data = all_positions[:, i, :][::-1]  # Reverse to get the latest first
        balloon_data_scaled = scalers[i].transform(balloon_data)
        latest_sequence = balloon_data_scaled[:time_steps].reshape(1, time_steps, 3)

        predicted_scaled = model.predict(latest_sequence, verbose=0)
        predicted_real = scalers[i].inverse_transform(predicted_scaled)
        predicted_next_real.append(predicted_real)

    return np.array(predicted_next_real)

# Predict the next time step
predicted_next_position = predict_next_position(model, all_positions, scalers, time_steps)

print("Predicted Next Positions (Lat, Lon, Alt):")
print(predicted_next_position)


LSTM Input Shape: (2000, 5, 3)
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


113/113 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.3026 - val_loss: 0.1386
Epoch 2/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0897 - val_loss: 0.0384
Epoch 3/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0365 - val_loss: 0.0298
Epoch 4/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0296 - val_loss: 0.0195
Epoch 5/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0208 - val_loss: 0.0174
Epoch 6/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0157 - val_loss: 0.0198
Epoch 7/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0137 - val_loss: 0.0143
Epoch 8/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0124 - val_loss: 0.0152
Epoch 9/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0125 - val_loss: 0.0180
Epoch 10/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0130 - val_loss: 0.0132
Epoch 11/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0119 - val_loss: 0.0123
Epoch 12/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0

In [8]:
template = """You are an operational analyst for a weather balloon company. Analyze the data of 
balloon positions over 24H: {all_positions}, LSTM prediction: {LSTM prediction}, pred next position: {pred next position}

Extract 3 key insights, such as:
- Spatial clusters indicating wind patterns
- Anomalies (e.g., balloons stuck in one area)
- Suggestions for optimizing future launches
Format the response as a bullet-point report."""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

# Generate insights
chain = prompt | model
report = chain.invoke({"all_positions": str(all_positions), "LSTM prediction": str(y_pred), "pred next position": str(predicted_next_position)})
print(report.content)

- Spatial clusters indicating wind patterns: By analyzing the data of balloon positions over 24 hours, we can identify spatial clusters that indicate wind patterns. These clusters may help in understanding the direction and intensity of winds at different altitudes.

- Anomalies: There do not seem to be any significant anomalies in the data, indicating that the balloons are not stuck in one particular area or facing any unexpected issues during the 24-hour period. This suggests that the balloon launches are functioning smoothly.

- Suggestions for optimizing future launches: Based on the LSTM predictions for the next positions of the balloons, we can optimize future launches by adjusting the launch locations or timings to minimize travel time and maximize data collection efficiency. Additionally, utilizing the spatial clusters identified in the data can help in planning more strategic launch locations to gather diverse weather data.


In [ ]:
# Save results
output = {
    "mse": mse,
    "r2": r2,
    "predicted_next_position": predicted_next_position.tolist(),
    "insights": report.content
}
with open("docs/output.json", "w") as f:
    json.dump(output, f, indent=4)

html_output = f"""
<html><head><title>Windborne Balloon Analysis</title></head>
<body>
    <h1>Windborne Balloon Analysis</h1>
    <p><strong>Mean Squared Error (MSE):</strong> {mse:.6f}</p>
    <p><strong>R² Score:</strong> {r2:.6f}</p>
    <h2>Predicted Next Position:</h2>
    <pre>{predicted_next_position.tolist()}</pre>
    <h2>Insights:</h2>
    <p>{report.content}</p>
</body></html>
"""
with open("docs/index.html", "w") as f:
    f.write(html_output)

print("✅ Output saved in docs/index.html and docs/output.json")
